<a href="https://colab.research.google.com/github/DanielCarrasquilla/Curso_basico_python/blob/master/Validacion412.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.2.1
!pip install findspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [4]:
spark

In [5]:
df = spark.read.format("csv").option("delimiter", "|").option("header","true").load("FORMATO_412_PROTO_20230304.csv")

In [6]:
df.show()

+---------+------------+-------------------+----------------+------------------------+--------------------------------+-----------------+
|subcuenta|codigo_canal|codigo_departamento|codigo_municipio|cantidad_puntos_atencion|cantidad_puntos_atencion_activos|unidad_de_captura|
+---------+------------+-------------------+----------------+------------------------+--------------------------------+-----------------+
|       23|          10|                 25|             572|                      14|                              14|               01|
|       24|          10|                 47|             745|                       2|                               2|               01|
|       25|          14|                 15|             572|                       1|                               1|               01|
|       26|          04|                 52|             835|                       6|                               6|               01|
|       27|          04|          

In [7]:
df.printSchema()

root
 |-- subcuenta: string (nullable = true)
 |-- codigo_canal: string (nullable = true)
 |-- codigo_departamento: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- cantidad_puntos_atencion: string (nullable = true)
 |-- cantidad_puntos_atencion_activos: string (nullable = true)
 |-- unidad_de_captura: string (nullable = true)



In [8]:
df.createTempView("df")

In [9]:
#Validacion de duplicados 412
spark.sql("""
select *
from df
where codigo_canal = '14'
and codigo_departamento = '13'
and codigo_municipio = '001'
""").show()

+---------+------------+-------------------+----------------+------------------------+--------------------------------+-----------------+
|subcuenta|codigo_canal|codigo_departamento|codigo_municipio|cantidad_puntos_atencion|cantidad_puntos_atencion_activos|unidad_de_captura|
+---------+------------+-------------------+----------------+------------------------+--------------------------------+-----------------+
|       83|          14|                 13|             001|                      13|                              13|               01|
+---------+------------+-------------------+----------------+------------------------+--------------------------------+-----------------+



In [10]:
spark.sql(""" 
select * from 
(select codigo_canal,codigo_departamento,codigo_municipio,count(*) as cantidad from df
group by codigo_canal,codigo_departamento,codigo_municipio) a
where a.cantidad != 1
""").show(n=1008)

+------------+-------------------+----------------+--------+
|codigo_canal|codigo_departamento|codigo_municipio|cantidad|
+------------+-------------------+----------------+--------+
+------------+-------------------+----------------+--------+



In [13]:
#Lectura Formato 413

df413 = spark.read.format("csv").option("delimiter", "|").option("header","true").load("FORMATO_413_PROTO_20230204(1).csv")

In [14]:
#Validación de duplicados 413
df413.createTempView("df413")

spark.sql(
"""
Select 
cod_distribucion , cod_serv_transaccional , cod_departamento , cod_municipio , cod_operacion , cod_tipo_persona , cod_ciiu , cod_mecanismo , unidad_de_captura,
count(*) from df413
GROUP BY 
cod_distribucion , cod_serv_transaccional , cod_departamento , cod_municipio , cod_operacion , cod_tipo_persona , cod_ciiu , cod_mecanismo , unidad_de_captura 
HAving count(*) > 1
"""
).show()


+----------------+----------------------+----------------+-------------+-------------+----------------+--------+-------------+-----------------+--------+
|cod_distribucion|cod_serv_transaccional|cod_departamento|cod_municipio|cod_operacion|cod_tipo_persona|cod_ciiu|cod_mecanismo|unidad_de_captura|count(1)|
+----------------+----------------------+----------------+-------------+-------------+----------------+--------+-------------+-----------------+--------+
+----------------+----------------------+----------------+-------------+-------------+----------------+--------+-------------+-----------------+--------+



In [15]:
spark.sql("select distinct(df413.cod_distribucion), df413.cod_serv_transaccional from df413 where df413.cod_operacion ='N/A'").show()

+----------------+----------------------+
|cod_distribucion|cod_serv_transaccional|
+----------------+----------------------+
|              02|                    00|
|              04|                    00|
|              01|                    00|
|              13|                    00|
|              14|                    00|
+----------------+----------------------+



In [22]:
df4=spark.sql("""
SELECT 
    subcuenta,
    CASE 
        WHEN cod_serv_transaccional = '00' THEN '00'
    END AS cod_distribucion,
    cod_serv_transaccional,
    cod_departamento,
    cod_municipio,
    cod_operacion,
    cod_tipo_persona,
    cod_ciiu,
    cod_mecanismo,
    cantidad_operaciones,
    monto_operaciones,
    unidad_de_captura
FROM df413
where cod_operacion ='N/A'
    """
)

In [ ]:
df4=spark.sql("""
SELECT 
    subcuenta,
    cod_distribucion,
    cod_serv_transaccional,
    CASE 
        WHEN cod_serv_transaccional = '00' THEN '00'
    END AS 
    cod_departamento,
    cod_municipio,
    cod_operacion,
    cod_tipo_persona,
    cod_ciiu,
    cod_mecanismo,
    cantidad_operaciones,
    monto_operaciones,
    unidad_de_captura
FROM df413
where cod_operacion ='N/A'
    """
)

In [19]:
df4.createTempView("df4")

In [21]:
spark.catalog.dropTempView("df4")

In [20]:
spark.sql("select * from df4").show()

+---------+----------------+----------------------+----------------+-------------+-------------+----------------+--------+-------------+--------------------+-----------------+-----------------+
|subcuenta|cod_distribucion|cod_serv_transaccional|cod_departamento|cod_municipio|cod_operacion|cod_tipo_persona|cod_ciiu|cod_mecanismo|cantidad_operaciones|monto_operaciones|unidad_de_captura|
+---------+----------------+----------------------+----------------+-------------+-------------+----------------+--------+-------------+--------------------+-----------------+-----------------+
|    11160|              00|                    00|              25|          899|          N/A|             N/A|   05012|          N/A|                   2|          1750000|               02|
|    11161|              00|                    00|              00|          000|          N/A|             N/A|    4690|          N/A|                   3|          3031900|               02|
|    11162|              00|  